<a href="https://colab.research.google.com/github/realnus/scikit_learn/blob/main/ModelSelectionUsingBayesOPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tpot

     |████████████████████████████████| 87 kB 6.6 MB/s 
     |████████████████████████████████| 160 kB 69.2 MB/s 
     |████████████████████████████████| 173.5 MB 70 kB/s 
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11952 sha256=18a08c0a41f838578bae806081962ba20d12a11a1bfae0f288a748dcbff32e77
  Stored in directory: /root/.cache/pip/wheels/e2/d2/79/eaf81edb391e27c87f51b8ef901ecc85a5363dc96b8b8d71e3
Successfully built stopit
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
import math

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

train_df = pd.read_csv("drive/MyDrive/VERİ/train.csv")
test_df=pd.read_csv("drive/MyDrive/VERİ/test.csv")
gender_sur=pd.read_csv("drive/MyDrive/VERİ/gender_submission.csv")

frames = [test_df, gender_sur]
test_df = pd.concat(frames, axis=1)

train_df = train_df.drop(['PassengerId', 'Cabin', 'Ticket'], axis=1)
test_df=test_df.drop(['PassengerId', 'Cabin', 'Ticket'], axis=1)

train_df['Title'] = train_df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
test_df['Title'] = test_df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(train_df['Title'], train_df['Sex'])

train_df['Title'] = train_df['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
train_df['Title'] = train_df['Title'].replace('Mlle', 'Miss')
train_df['Title'] = train_df['Title'].replace('Ms', 'Miss')
train_df['Title'] = train_df['Title'].replace('Mme', 'Mrs')

test_df['Title'] = test_df['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
test_df['Title'] = test_df['Title'].replace('Mlle', 'Miss')
test_df['Title'] = test_df['Title'].replace('Ms', 'Miss')
test_df['Title'] = test_df['Title'].replace('Mme', 'Mrs')

title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
train_df['Title'] = train_df['Title'].map(title_mapping)
train_df['Title'] = train_df['Title'].fillna(0)

test_df['Title'] = test_df['Title'].map(title_mapping)
test_df['Title'] = test_df['Title'].fillna(0)

train_df = train_df.drop(['Name'], axis=1)
test_df = test_df.drop(['Name'], axis=1)

train_df['Sex'] = train_df['Sex'].map( {'female': 1, 'male': 0} ).astype(int)
test_df['Sex'] = test_df['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

train_df['Age']=train_df['Age'].astype(int)
test_df['Age']=test_df['Age'].astype(int)

train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

combine= [train_df, test_df]
for dataset in combine:
    dataset.loc[dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[dataset['Age'] > 64, 'Age']=4

train_df = train_df.drop(['AgeBand'], axis=1)
combine = [train_df, test_df]

for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

train_df = train_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
test_df = test_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
combine = [train_df, test_df]

for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)

for dataset in combine:
    dataset['Age*Class'] = dataset.Age * dataset.Pclass

for dataset in combine:
    dataset.loc[ dataset['Fare'] <=8.05, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 8.05) & (dataset['Fare'] <= 15.646), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 15.646) & (dataset['Fare'] <=  33.0), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 33.0, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train_df = train_df.drop(['FareBand'], axis=1)
combine = [train_df, test_df]

X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("Survived", axis=1)
Y_test  = test_df["Survived"]

In [ ]:
models = []
models.append(('Logistic Regression', LogisticRegression(random_state=0)))
models.append(('Naive Bayes', GaussianNB()))
models.append(('Decision Tree (CART)',DecisionTreeClassifier(random_state=0)))
models.append(('K-NN', KNeighborsClassifier()))
models.append(('SVM', SVC(random_state=0)))
models.append(('Gradient Boosting Classifier', GradientBoostingClassifier(random_state=0)))
models.append(('AdaBoostClassifier', AdaBoostClassifier(random_state=0)))
models.append(('BaggingClassifier', BaggingClassifier(random_state=0)))
models.append(('RandomForestClassifier', RandomForestClassifier(random_state=0)))
models.append(("XGBootsClassifier",xgb.XGBClassifier(random_state=0,eval_metric='mlogloss',use_label_encoder =False)))


from sklearn.metrics import accuracy_score

for name, model in models:
    model = model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    y_true = Y_test
    y_pred_train= model.predict(X_train)
    y_true_train=Y_train

    print("%s -> TEST ACC: %%%.2f     TRAİN ACC: %%%.2f" % (name,accuracy_score(y_true, y_pred)*100,accuracy_score(y_pred_train,y_true_train)*100))

Logistic Regression -> TEST ACC: %93.05     TRAİN ACC: %80.06
Naive Bayes -> TEST ACC: %81.27     TRAİN ACC: %76.26
Decision Tree (CART) -> TEST ACC: %83.08     TRAİN ACC: %87.36
K-NN -> TEST ACC: %82.18     TRAİN ACC: %85.11
SVM -> TEST ACC: %92.45     TRAİN ACC: %81.74
Gradient Boosting Classifier -> TEST ACC: %79.15     TRAİN ACC: %85.53
AdaBoostClassifier -> TEST ACC: %85.80     TRAİN ACC: %80.62
BaggingClassifier -> TEST ACC: %82.48     TRAİN ACC: %87.36
RandomForestClassifier -> TEST ACC: %83.38     TRAİN ACC: %87.36
XGBootsClassifier -> TEST ACC: %82.78     TRAİN ACC: %87.22


In [ ]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

#hyperopt:Bir hiperparametre değer uzayında arama yapan güçlü bir python kitaplığıdır.
#Maliyet işlevini en aza indirmek için üç işlevi yerine getirir


search_space = hp.choice('classifier_type', [
    {
        'type': 'svm',
        "kernel": hp.choice("kernel",["linear","poly","rbf","sigmoid"]),
        'gamma': hp.choice("gamma1",["scale","auto"]+[math.pow(10,i) for i in range(-5,0,1)]),
        "C":hp.choice("C",np.arange(0.001,50,0.010)),
        "shrinking":hp.choice("shrinking",[True,False]),
        "probability":hp.choice("probability",[True,False]),
        "tol":hp.choice("tol",[math.pow(10,i) for i in range(-5,0,1)]),
        "decision_function_shape":hp.choice("decision_function_shape",["ovo","ovr"]),
        "random_state":hp.choice("random_state1",[*range(1,102,1)])
    },
    {
        'type': 'rf',
        'criterion': hp.choice('criterion', ['entropy','gini']), #best practice values -DK 15.09.2021
        'max_features': hp.choice('max_features', [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,'auto', 'sqrt','log2',None]), #best practice values -DK 15.09.2021
        'n_estimators' : hp.choice('n_estimators1', [*range(10,1010,10)]), #best practice values -DK 15.09.2021
        'bootstrap': hp.choice('bootsrap',[True,False]), #best practice values -DK 15.09.2021
        'class_weight': hp.choice('class_weight',["balanced","balanced_subsample",None]), #best practice values -DK 15.09.2021
        "min_samples_leaf":hp.choice("min_samples_leaf",[*range(1,21,1)]), #best practice values -DK 15.09.2021
        "min_samples_split":hp.choice("min_samples_split",[*range(2,21,1)]), #best practice values -DK 15.09.2021
        "max_depth":hp.choice("max_depth1",[*range(5,55,5)]+[None]), #best practice values -DK 15.09.2021
        "random_state":hp.choice("random_state2",[*range(1,102,1)]) #best practice values -DK 15.09.2021
    },
    {
        'type': 'xgb',
        'booster': hp.choice('booster', ['gbtree','dart']), #best practice values -DK 22.09.2021
        'colsample_bytree': hp.choice('colsample_bytree', np.arange(0, 1,0.1)), #best practice values -DK 22.09.2021
        'gamma' :hp.choice('gamma2',np.arange(0,5,0.1)), #best practice values -DK 22.09.2021
        'learning_rate': hp.choice('learning_rate',np.arange(0.001,1,0.001)), #best practice values -DK 22.09.2021
        "max_depth":hp.choice("max_depth2",[*range(5,115,5)]+[1]), #best practice values -DK 22.09.2021
        "min_child_weight":hp.choice("min_child_weight",[*range(0,21,1)]), #best practice values -DK 22.09.2021
        'n_estimators' : hp.choice('n_estimators2', [*range(10,1010,10)]), #best practice values -DK 22.09.2021
        'objective': hp.choice('objective',["binary:logistic","binary:logitraw","binary:hinge"]), #best practice values -DK 22.09.2021
        "random_state":hp.choice('random_state3', [*range(0,102,1)]), #best practice values -DK 22.09.2021
        "reg_alpha":hp.choice('reg_alpha', [*range(0,16,1)]), #best practice values -DK 22.09.2021
        "reg_lambda":hp.choice('reg_lambda',[*range(0,16,1)]), #best practice values -DK 22.09.2021
        "eval_metric":hp.choice("eval_metric",["rmse","mape","rmsle","mae","mphe","logloss","error","error@0.5","merror","mlogloss","auc","aucpr","ndcg","map","ndcg@0.5","map@0.5","ndcg-","map-","ndcg@0.5-","map@0.5-","poisson-nloglik","gamma-nloglik","cox-nloglik","gamma-deviance","aft-nloglik","interval-regression-accuracy"]),
        "use_label_encoder":hp.choice("use_label_encoder",[False])
    },
])

In [ ]:
!pip install mlflow

In [ ]:
import mlflow

In [ ]:
from sklearn.model_selection import cross_val_score

def objective(params):
    classifier_type = params['type']
    del params['type']
    if classifier_type == 'svm':
        clf = SVC(**params)
    elif classifier_type == 'rf':
        clf = RandomForestClassifier(**params)
    elif classifier_type == 'xgb':
        clf = xgb.XGBClassifier(**params)
    else:
      return 0
    cv = StratifiedKFold(n_splits=4,random_state=42,shuffle=True)
    accuracy = cross_val_score(clf, X_train, Y_train, cv = cv).mean()

    # Because fmin() tries to minimize the objective, this function must return the negative accuracy.
    return {'loss': -accuracy, 'status': STATUS_OK}

In [ ]:
acc=[]
for i in range(5):
  with mlflow.start_run():
    int_rand=np.random.randint(0,2**32-1)
    rstate = np.random.RandomState(int_rand)
    trials = Trials()
    best=fmin(fn= objective,space=search_space,algo= tpe.suggest,max_evals =100,trials= trials,rstate=rstate)
    model_dict=hyperopt.space_eval(search_space, best)
    type=model_dict["type"]
    del model_dict["type"]
    acc.append((type,model_dict,int_rand))

100%|██████████| 100/100 [05:43<00:00,  3.44s/it, best loss: -0.827247191011236]


In [ ]:
from sklearn.metrics import accuracy_score
best_acc=[]

for classifier_type,params,int_rand in acc:
  if classifier_type== 'svm':
    clf = SVC(**params)
  elif classifier_type == 'rf':
    clf = RandomForestClassifier(**params)
  elif classifier_type == 'xgb':
    clf = xgb.XGBClassifier(**params)
  trainedforest = clf
  trainedforest.fit(X_train,Y_train)
  Y_pred = trainedforest.predict(X_test)
  Y_true=Y_test
  Y_pred_train= trainedforest.predict(X_train)
  Y_true_train=Y_train
  acc_random_forest =accuracy_score(Y_true, Y_pred)*100
  acc_train_random=accuracy_score(Y_pred_train, Y_true_train)*100
  best_acc.append((classifier_type,acc_random_forest,acc_train_random,trainedforest.get_params,int_rand))

In [ ]:
for classifier_type,test_acc,train_acc,parameters,int_rand in best_acc:
  print("CLASSİFİER TYPE",classifier_type,"","TEST ACC: ",test_acc,"","TRAİN ACC: ",train_acc,"","PARAMS: ",parameters,"","INTEGER RAND: ",int_rand)

CLASSİFİER TYPE rf  TEST ACC:  80.06042296072508  TRAİN ACC:  86.65730337078652  PARAMS:  <bound method BaseEstimator.get_params of RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                       class_weight='balanced_subsample', criterion='gini',
                       max_depth=20, max_features='log2', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=5, min_weight_fraction_leaf=0.0,
                       n_estimators=450, n_jobs=None, oob_score=False,
                       random_state=15, verbose=0, warm_start=False)>  INTEGER RAND:  1630613758
CLASSİFİER TYPE svm  TEST ACC:  81.2688821752266  TRAİN ACC:  86.65730337078652  PARAMS:  <bound method BaseEstimator.get_params of SVC(C=42.54099999999999, break_ties=False, cache_size=200, class_weight=None,
    coef0=0.0, decision_function_shape='ovo', degree=3, gamma=0.1, kern